# Klasy

In [23]:
class Gadget:
    def __init__(self, id: int, name: str) -> None:
        self.id = id
        self.name = name
    
    def use(self) -> None:
        print(f"Using {self.name} with Id#{self.id}")    
        self.use_count = 1

    def __repr__(self) -> str:
        return repr(self.__dict__)
        
    
    def __str__(self) -> str:
        return f"Gadget({self.id}, {self.name})"

In [24]:
my_gadget = Gadget(1, "ipad")
my_gadget.use()

Using ipad with Id#1


In [25]:
my_gadget

{'id': 1, 'name': 'ipad', 'use_count': 1}

In [19]:
eval(repr(my_gadget))

Gadget(id=1, name='ipad')

In [20]:
print(my_gadget)

Gadget(1, ipad)


In [22]:
my_gadget

Gadget(id=1, name='ipad')

## Metody `__new__` i `__init__`

In [26]:
class Dummy:
    def __new__(cls, *args):
        print(f"Dummy.__new__({cls}, {args}) has been called...")
        obj = super().__new__(cls)
        obj.extra_attribute = "extra"
        print(f"Object {obj} has been created...")
        return obj
    
    def __init__(self, *args):
        print(f"Dummy.__init__({self}, {args})...")
        self.args = args
        print(f"Object's __dict__: {self.__dict__}")


In [27]:
d = Dummy(1, "two")

Dummy.__new__(<class '__main__.Dummy'>, (1, 'two')) has been called...
Object <__main__.Dummy object at 0x0000024297785850> has been created...
Dummy.__init__(<__main__.Dummy object at 0x0000024297785850>, (1, 'two'))...
Object's __dict__: {'extra_attribute': 'extra', 'args': (1, 'two')}


## Kiedy używamy `__new__`?

In [28]:
class UppercaseTuple(tuple):
    def __init__(self, list) -> None:
        print(f"Start changes for {list}")
        for i, item in enumerate(list):
            self[i] = item.upper()


In [29]:
UppercaseTuple(["hello", "world"])

Start changes for ['hello', 'world']


TypeError: 'UppercaseTuple' object does not support item assignment

In [46]:
class UppercaseTuple(tuple):
    def __new__(cls, list):
        print(f"Start changes for {list}")
        new_content = [item.upper() for item in list]       
        return super().__new__(cls, new_content)

In [47]:
ut = UppercaseTuple(["hello", "world"])

Start changes for ['hello', 'world']


In [48]:
ut

('HELLO', 'WORLD')

## `__getattr__` , `__setattr__` & `__delattr__`

In [59]:
class Record:

    def __init__(self):
        # Nie możemy użyć poniższego kodu:
        #     self._d = {}
        # ponieważ zakończyłby się on rekurencyjnym wywoływaniem metody __setattr__
        super().__setattr__('_dict', {})

    def __getattr__(self, name):
        print('getting', name)
        return self._dict[name]
    
    def __setattr__(self, name, value):
        print('setting', name, 'to', value)
        self._dict[name] = value
        
    def __delattr__(self, name):
        print('deleting', name)
        del self._dict[name]

In [60]:
r1 = Record()

In [61]:
r1.first_name = 'John'

In [62]:
r1.__dict__

{'first_name': 'John'}

In [52]:
r1.first_name

getting first_name


'John'

In [53]:
r1.first_name += ' F.'

getting first_name
setting first_name to John F.


In [54]:
del r1.first_name

deleting first_name


## `__getattribute__`

In [63]:
class Person:
    def __init__(self, first_name):
        self.first_name = first_name
    
    def __getattribute__(self, name):
        print('getattribute', name)
        return object.__getattribute__(self, name)

In [64]:
p = Person("John")

In [65]:
p.__dict__

getattribute __dict__


{'first_name': 'John'}

In [66]:
p.first_name

getattribute first_name


'John'

In [10]:
import datetime


class Foo:
    def __init__(self):
        self.a = "a"

    def __getattr__(self, attribute):
        return f"You asked for {attribute}, but I'm giving you default"


class Bar(Foo):
    attribute_access_log = []

    def __init__(self):
        self.a = "a"
        self.access_log = []

    def __getattribute__(self, attribute):
        Bar.attribute_access_log.append(
            f"Access to {self}.{attribute} at {datetime.datetime.now()}")
        print(f"You asked for {attribute}")
        return super().__getattribute__(attribute)

In [12]:
bar = Bar()
bar.a

You asked for a


'a'

In [13]:
bar.a

You asked for a


'a'

In [14]:
Bar.attribute_access_log

['Access to <__main__.Bar object at 0x00000238ABF26990>.a at 2023-04-17 11:59:54.452290',
 'Access to <__main__.Bar object at 0x00000238ABF26990>.a at 2023-04-17 11:59:55.923565']

## Składowe prywatne

In [22]:
class BankAccount:
    def __init__(self, init_balance: float) -> None:
        self._balance = init_balance

    @property # getter
    def balance(self) -> float:
        return self._balance

    @balance.setter
    def balance(self, value: float) -> None:
        print(f'Setting balance for {self} to {value}')
        self._balance = value
        

In [23]:
account_1 = BankAccount(1000)
account_1.balance

1000

In [24]:
account_1.balance = 3000

Setting balance for <__main__.BankAccount object at 0x00000238ABFEE410> to 3000


In [25]:
BankAccount.balance

## staticmethod & classmethod

In [26]:
class Date:
    def __init__(self, day, month, year):
        self.day = day
        self.month = month
        self.year = year
    
    @classmethod
    def from_string(cls, date_as_string):
        print(f"Calling from_string for class: {cls}")
        day, month, year = date_as_string.split('-')
        return cls(int(day), int(month), int(year)) # utworzenie instancji klasy cls

In [27]:
Date.from_string('10-04-2023')

Calling from_string for class: <class '__main__.Date'>


In [28]:
date1 = Date.from_string('10-04-2023')

Calling from_string for class: <class '__main__.Date'>


In [29]:
date1.from_string('22-02-2023')

Calling from_string for class: <class '__main__.Date'>


In [33]:
class CountedObject(object):
    count = 0   # statyczna składowa
    
    def __init__(self):
        CountedObject.count += 1
    
    @staticmethod  # statyczna metoda
    def get_count():
        return CountedObject.count

In [34]:
CountedObject.count

0

In [35]:
c1 = CountedObject()
c2 = CountedObject()

In [39]:
CountedObject.get_count()

2

In [42]:
class Person:
    first_name: str = "unknown"

    def __init__(self, fname = None):
        if fname is not None:
            self.first_name = fname

In [43]:
p1 = Person()
p1.first_name

'unknown'

In [44]:
p2 = Person("John")
p2.first_name

'John'

## Deskryptory

In [46]:
class Ten:
    def __get__(self, obj, owner = None):
        print(f"Ten.__get__({self}, {obj}, {owner})")
        return 10

class Const:
    value = Ten()

In [47]:
const = Const()

In [49]:
const.value

Ten.__get__(<__main__.Ten object at 0x00000238ABFF7690>, <__main__.Const object at 0x00000238AC560C90>, <class '__main__.Const'>)


10

In [51]:
import os

class DirectorySize:
    """Non-data descriptor"""

    def __get__(self, obj, objtype=None):
        print("getting directory size")
        return len(os.listdir(obj.dirname))

class Directory:
    size = DirectorySize()              # Descriptor instance

    def __init__(self, dirname):
        self.dirname = dirname          # Regular instance attribute

In [52]:
local_dir = Directory('.')

In [53]:
local_dir.__dict__

{'dirname': '.'}

In [54]:
local_dir.size

getting directory size


1

### Walidacja danych z deskryptorem

In [69]:
from typing import Callable, Any

class ValidatedAttribute:
    """Data descriptor"""
    def __init__(self, validator: Callable[..., bool]) -> None:
        self.validator = validator

    def __set_name__(self, owner: Any, name: str):
        self.private_name = '_ValidatedAttribute__' + name
        print(f'Setting private name to: {self.private_name}')

    def __get__(self, obj: Any, owner: type):
        print("ValidatedAttribute.__get__")
        print(f"..........self: {self}")
        print(f"......instance: {obj}")
        print(f".........owner: {owner}")
        return getattr(obj, self.private_name) # obj._even

    def __set__(self, obj, value):
        print("ValidatedAttribute.__set__")
        print(f"..........self: {self}")
        print(f"......instance: {obj}")
        print(f".........value: {value}")
        if self.validator(value):
            setattr(obj, self.private_name, value)
        else:
            raise AttributeError("Attribute validation failed")

In [70]:
class Data:
    even = ValidatedAttribute(lambda n: n % 2 == 0)

    def __init__(self, value):
        self._even = 665
        self.even = value

Setting private name to: _ValidatedAttribute__even


In [71]:
d1 = Data(64)

ValidatedAttribute.__set__
..........self: <__main__.ValidatedAttribute object at 0x00000238AC55ABD0>
......instance: <__main__.Data object at 0x00000238AC5B5350>
.........value: 64


In [72]:
d1.__dict__

{'_even': 665, '_ValidatedAttribute__even': 64}

In [73]:
d1.even

ValidatedAttribute.__get__
..........self: <__main__.ValidatedAttribute object at 0x00000238AC55ABD0>
......instance: <__main__.Data object at 0x00000238AC5B5350>
.........owner: <class '__main__.Data'>


64

In [74]:
d1.even = 13

ValidatedAttribute.__set__
..........self: <__main__.ValidatedAttribute object at 0x00000238AC55ABD0>
......instance: <__main__.Data object at 0x00000238AC5B5350>
.........value: 13


AttributeError: Attribute validation failed

In [64]:
d1.even

ValidatedAttribute.__get__
..........self: <__main__.ValidatedAttribute object at 0x00000238AC00A790>
......instance: <__main__.Data object at 0x00000238ABFFBD50>
.........owner: <class '__main__.Data'>


64

In [66]:
d1._even

64

### Invocation chain in Python

#### Invocation from an instance:

`object.__getattribute__()` transforms `b.x` into `type(b).__dict__['x'].__get__(b, type(b))`

Let’s say we are looking for attribute `x` on object `o` (`x.o`):

* **data descriptors**: value from `__get__` method of the data descriptor named after x

* **instance variables**: value of `o. __dict__` for the key named as `x`

* **non-data descriptors**: value from `__get__` method of the non-data descriptor named after `x`

* **class variables**: `type(o).__dict__` for the key named as `x`

* **parent’s class variables** all the way along the MRO,

* **`__getattr__()`** if it is provided.

#### Invocation from a class

* `type.__getattribute__()` which transforms `B.x` into `B.__dict__['x'].__get__(None, B)` .

* The logic for a dotted lookup such as `A.x` is in `type.__getattribute__()`. The steps are similar to instance dictionary lookup but it’s a search through the class’s method resolution order.

* If a descriptor is found, it is invoked with `desc.__get__(None, A)`

## Dziedziczenie

In [81]:
class Base:
    def __init__(self, value):
        self.value = value

    def base_method(self):
        pass

class Derived(Base):
    def __init__(self, value, derived_value):
        super().__init__(value)
        self.derived_value = derived_value
    
    def derived_method(self):
        pass

In [82]:
d = Derived(42, 665)

In [83]:
isinstance(d, Base)

True

In [84]:
d.__dict__

{'value': 42, 'derived_value': 665}

In [87]:
Base.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Base.__init__(self, value)>,
              'base_method': <function __main__.Base.base_method(self)>,
              '__dict__': <attribute '__dict__' of 'Base' objects>,
              '__weakref__': <attribute '__weakref__' of 'Base' objects>,
              '__doc__': None})

In [86]:
Derived.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Derived.__init__(self, value, derived_value)>,
              'derived_method': <function __main__.Derived.derived_method(self)>,
              '__doc__': None})

In [88]:
Derived.__mro__

(__main__.Derived, __main__.Base, object)

## Abstract Base Classes

In [95]:
from abc import ABC, abstractmethod

class IBankAccount(ABC):
    @abstractmethod
    def withdraw(self, amount: float) -> None:
        pass

    @abstractmethod
    def deposit(self, amount: float) -> None:
        pass

In [107]:
class BankAccount(IBankAccount):
    def __init__(self, initial_balance: float, daily_limit: float) -> None:
        self.__balance = initial_balance
        self.__daily_limit = daily_limit
    
    @property
    def balance(self) -> float:
        print('getting balance')
        return self.__balance

    @property
    def daily_limit(self) -> float:
        print('getting daily_limit')
        return self.__daily_limit
    
    @daily_limit.setter
    def daily_limit(self, value: float) -> None:
        if value < 0:
            raise ValueError('Value must be >= 0')
        self.__daily_limit = value


In [114]:
class DebitAccount(BankAccount):
    def __init__(self, initial_balance: float, daily_limit: float, debit_limit: float) -> None:
        super().__init__(initial_balance, daily_limit)
        self.__debit_limit = debit_limit

    @property
    def debit_limit(self) -> float:
        return self.__debit_limit
    
    @debit_limit.setter
    def debit_limit(self, limit: float) -> None:
        self.__debit_limit = limit

    def withdraw(self, amount: float) -> None:
        self._BankAccount__balance -= amount

    def deposit(self, amount: float) -> None:
        self._BankAccount__balance += amount
    

In [115]:
account = DebitAccount(1000, 200, 500)

In [116]:
account.balance

getting balance


1000

In [117]:
account.debit_limit

500

In [118]:
account.withdraw(100)

In [119]:
account.balance

getting balance


900

In [121]:
account2 = BankAccount(1000, 400)

TypeError: Can't instantiate abstract class BankAccount with abstract methods deposit, withdraw

### Dziedziczenie wielobazowe

In [122]:
class A:
    def __init__(self):
        print("A")

class B(A):
    def __init__(self):
        super().__init__()
        print("B")

class C(A):
    def __init__(self):
        super().__init__()
        print("C")

class D(B, C):
    def __init__(self):
        super().__init__()
        print("D")

In [123]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

## Mixins

In [141]:
class ComparableMixin:
    def __ne__(self, other):
        return not (self == other)
    
    def __lt__(self, other):
        return self <= other and (self != other)
    
    def __gt__(self, other):
        return not self <= other
    
    def __ge__(self, other):
        return self == other or self > other

class Value(ComparableMixin):
    def __init__(self, value):
        self.value = value

    def __eq__(self, other):
        return self.value == other.value

    def __le__(self, other):
        return self.value <= other.value

In [142]:
v1 = Value(1)
v2 = Value(1)

In [143]:
v1 == v2

True

In [144]:
v1 > Value(0)

True

In [145]:
import functools

@functools.total_ordering
class Value:
    def __init__(self, value):
        self.value = value

    def __eq__(self, other):
        return self.value == other.value

    def __lt__(self, other):
        return self.value < other.value
    

In [146]:
Value(10) >= Value(5)

True

## abc.Sequence Mixin

In [147]:
from collections.abc import Sequence

class VerboseTuple(Sequence):
    """Custom class that is exactly like a tuple but does some
    extra magic.

    Sequence:
    -------------------
    Inherits From: Reversible, Collection
    Abstract Methods: __getitem__, __len__
    Mixin Methods: __contains__, __iter__, __reversed__, index,
            and count
    """

    def __init__(self, *args):
        self.args = args

    @classmethod
    def _classname(cls):
        # This method just returns the name of the class
        return cls.__name__
    
    def __getitem__(self, index):
        print(f"Method: __getitem__, Index: {index}")
        return self.args[index]

    def __len__(self):
        print(f"Method: __len__")
        return len(self.args)

    def __repr__(self):
        return f"{self._classname()}{tuple(self.args)}"

In [149]:
vt = VerboseTuple(1, 2, 3)
vt

VerboseTuple(1, 2, 3)

In [150]:
print(f"Abstract Methods: {set(Sequence.__abstractmethods__)}")
print(f"Mixin Methods: { {k for k, v in Sequence.__dict__.items() if callable(v)} }")

Abstract Methods: {'__getitem__', '__len__'}
Mixin Methods: {'__reversed__', 'index', '__getitem__', '__contains__', 'count', '__iter__'}


In [151]:
len(vt)

Method: __len__


3

In [153]:
tuple(reversed(vt))

Method: __len__
Method: __getitem__, Index: 2
Method: __getitem__, Index: 1
Method: __getitem__, Index: 0


(3, 2, 1)

In [154]:
vt.index(2)

Method: __getitem__, Index: 0
Method: __getitem__, Index: 1


1

In [156]:
1 in vt

Method: __getitem__, Index: 0


True

In [157]:
id(1)

140731597517608

In [159]:
id(1237654127635476234581237645281374523768154812376457682345823645)

2442434991984